In [1]:
from sklearn.datasets import load_iris
import pandas as pd
from preprocessing import preprocess_data
from joblib import dump, load
import numpy as np
 
# Memuat dataset
iris = load_iris()
data = pd.DataFrame(iris.data, columns=iris.feature_names)
data['target'] = iris.target
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [2]:
# Contoh Penggunaan
X_train, X_test, y_train, y_test = preprocess_data(data, 'target', 'preprocessor_pipeline.joblib', 'data.csv')

Nama kolom berhasil disimpan ke: data.csv


In [3]:
def inference(new_data, load_path):
    # Memuat pipeline preprocessing
    preprocessor = load(load_path)
    print(f"Pipeline preprocessing dimuat dari: {load_path}")
 
    # Transformasi data baru
    transformed_data = preprocessor.transform(new_data)
    return transformed_data

In [4]:
# Jalankan preprocessing
pipeline_path = 'preprocessor_pipeline.joblib'
col = pd.read_csv('data.csv')
# Daftar data
new_data = [5.1, 3.5, 1.4, 0.2]
 
# Mengubah menjadi numpy.ndarray
new_data = np.array(new_data)
 
new_data = pd.DataFrame([new_data], columns=col.columns)
# Lakukan inference
transformed_data = inference(new_data, pipeline_path)

Pipeline preprocessing dimuat dari: preprocessor_pipeline.joblib


In [5]:
def inverse_transform_data(transformed_data, load_path, new_data_columns):
    preprocessor = load(load_path)
    # Membalik transformasi hanya untuk fitur numerik
    numeric_transformer = preprocessor.named_transformers_['num']['scaler']
    numeric_columns = new_data_columns[:len(numeric_transformer.mean_)]  # Asumsi fitur numerik ada di awal
    transformed_numeric_data = transformed_data[:, :len(numeric_columns)]

    # Inverse transform untuk data numerik
    original_numeric_data = numeric_transformer.inverse_transform(transformed_numeric_data)

    # Gabungkan kembali data numerik dengan data kategoris (jika ada)
    inversed_data = pd.DataFrame(original_numeric_data, columns=numeric_columns)
    return inversed_data

In [6]:
# Inverse transform data
inversed_data = inverse_transform_data(transformed_data, pipeline_path, new_data.columns)

# Output hasil preprocessing dan inference
print("Data setelah preprocessing (training):")
print(new_data)
print("\nData baru setelah transformasi:")
print(transformed_data)
print("\nData setelah inverse transform:")
print(inversed_data)

Data setelah preprocessing (training):
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2

Data baru setelah transformasi:
[[-0.89573553  1.17645543 -1.44207638 -1.40568508]]

Data setelah inverse transform:
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
